In [1]:
!nvidia-smi

zsh:1: command not found: nvidia-smi


In [2]:
!pip install missingpy

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_auc_score
import math

In [7]:
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base

In [8]:
import sys
import numpy as np
import pandas as pd

from missingpy import MissForest
from dsan import Imputer as DSANImputer
from utils import *

import warnings
warnings.filterwarnings('ignore')

In [9]:
seed = 128
set_seed(seed)

In [10]:
def get_imputer(config):
    
    model = config['model']
    
    if model == 'DSAN':
        imputer = DSANImputer(rep_dim=config['rep_dim'],
                num_heads=config['num_heads'],
                n_hidden=config['n_hidden'],
                lr=config['lr'],
                weight_decay=config['weight_decay'],
                batch_size=config['batch_size'],
                epochs=config['epochs'],
                noise_percent=config['noise_percent'],
                stopped_epoch=config['stopped_epoch'])
    
    elif model == 'MissForest':
        imputer = MissForest(random_state=0)
        
    elif model == 'Statistics':
        imputer = None
        
    return imputer

In [11]:
def experiment_run(data_name, config, test=False):
    # Retrieve dataset
    dataset = get_data(data_name)
    print(f"Retrieved dataset for {data_name}")

    # Limit to 1000 samples if test flag is true
    if test:
        dataset = (dataset[0][:1000], dataset[1], dataset[2], dataset[3])
        print("Limited dataset to 1000 samples for testing")

    k = 5
    percent_missing_lst = [p for p in range(5, 25, 5)]
    X_origin, n_col, num_vars, cat_vars = dataset

   # Apply category mapping
    X_origin = category_mapping(X_origin, cat_vars)
    print("Applied category mapping")

    # Check if X_origin is modified in-place or replaced
    if X_origin is not None:
        print("X_origin is replaced with a new version where categorical variables have been mapped to numerical values.")
    else:
        print("X_origin is modified in-place, with categorical variables mapped to numerical values.")

    # Print data size
    print("Data Size: {}".format(X_origin.shape))


    n_sample = X_origin.shape[0]
    idx_lst = list(range(n_sample))
    np.random.shuffle(idx_lst)

    # k-fold validation setup
    n_valid = len(idx_lst) // k
    print(f"Starting k-fold validation with k={k}")

    results = {}
    for percent_missing in percent_missing_lst:
        result = []
        print(f"\nProcessing {percent_missing}% missing data")

        for it, n in enumerate(range(0, n_sample, n_valid)):
            print(f"\nFold {it + 1}")
            valid_idx = idx_lst[n: n+n_valid]

            if len(valid_idx) < n_valid:
                print("Insufficient data for validation, breaking loop")
                break

            train_idx = list(set(idx_lst) - set(valid_idx))
            X = X_origin[train_idx, :]
            X_test = X_origin[valid_idx, :]

            # Generating incomplete data
            missing_mask = generate_missing_mask(X, percent_missing=percent_missing)
            X_incomplete = X.copy()
            X_incomplete[missing_mask] = np.nan
            print("Generated incomplete data with missing values for fold {}:".format(it+1))
            print("Original data with missing values (subset):")
            print(X_incomplete[:5])  # Adjust the number of rows as needed

            # Imputation
            imputer = get_imputer(config)
            X_imputed = imputer.fit_transform(X_incomplete, cat_vars=cat_vars)
            print("Completed imputation for fold {}:".format(it+1))

            X_res = X.copy()
            X_res[missing_mask] = X_imputed[missing_mask]
            print("X_res with imputed values (subset):")
            print(X_res[:5])  # Adjust the number of rows as needed

            # Calculate metrics
            metric = cal_metric_numpy(X_res, X, missing_mask, num_vars, cat_vars)
            score = ex_classify(data_name=data_name, train_array=X_res, test_array=X_test, num_vars=num_vars, cat_vars=cat_vars)
            
            metric['clf_aucroc'] = round(score, 4)
            result.append(metric)
            print("Metrics for this fold:", metric)

        # Average results
        avg_result = dict()
        for key in result[0].keys():
            value = np.mean([m[key] for m in result])
            avg_result[key] = value 
            print(f"Average {key} for {percent_missing}% missing data: {value}")

        results[percent_missing] = avg_result

    print("Completed all iterations")
    return results


In [12]:
config = {
    'model': 'DSAN',
    'rep_dim': 32,
    'num_heads': 8,
    'n_hidden': 2,
    'lr': 3e-3,
    'weight_decay': 1e-5,
    'batch_size': 128,
    'epochs': 15,
    'noise_percent': 10,
    'stopped_epoch': 10
}

In [13]:
data_name_lst = ['titanic']

result_dir = './results_titanic'
os.makedirs(result_dir)

for data_name in data_name_lst:
    results = experiment_run(data_name, config, test=True)
    results['config'] = config
    results['dataset'] = data_name
    result_path = os.path.join(result_dir, '{}.json'.format(data_name))
    write_json(results, result_path)

Retrieved dataset for titanic
Limited dataset to 1000 samples for testing
Applied category mapping
X_origin is replaced with a new version where categorical variables have been mapped to numerical values.
Data Size: (887, 7)
Starting k-fold validation with k=5

Processing 5% missing data

Fold 1
Generated incomplete data with missing values for fold 1:
Original data with missing values (subset):
[[1 0 1 26 0 0 8]
 [1 1 1 35 1 0 53]
 [0 0 0 35 0 0 8]
 [0 0 0 27 0 nan 8]
 [0 nan 0 54 0 0 52]]


100%|███████████████████████████████████████████| 15/15 [00:03<00:00,  4.18it/s]


Completed imputation for fold 1:
X_res with imputed values (subset):
[[1 0 1 26 0 0 8]
 [1 1 1 35 1 0 53]
 [0 0 0 35 0 0 8]
 [0 0 0 27 0 1.7759735584259033 8]
 [0 1 0 54 0 0 52]]
Metrics for this fold: {'nrmse': 0.4599, 'col_0_error_rate': 0.1818, 'col_1_error_rate': 0.3636, 'col_2_error_rate': 0.4375, 'total_error_rate': 0.3303, 'clf_aucroc': 0.8603}

Fold 2
Generated incomplete data with missing values for fold 2:
Original data with missing values (subset):
[[nan 0 0 22 1 0 7]
 [1 1 1 38 1 0 71]
 [1 0 nan 26 0 0 8]
 [1 1 1 nan 1 0 53]
 [0 0 0 35 0 0 8]]


100%|███████████████████████████████████████████| 15/15 [00:03<00:00,  4.02it/s]


Completed imputation for fold 2:
X_res with imputed values (subset):
[[0 0 0 22 1 0 7]
 [1 1 1 38 1 0 71]
 [1 0 0 26 0 0 8]
 [1 1 1 28.40114402770996 1 0 53]
 [0 0 0 35 0 0 8]]
Metrics for this fold: {'nrmse': 1.3549, 'col_0_error_rate': 0.2581, 'col_1_error_rate': 0.4186, 'col_2_error_rate': 0.3571, 'total_error_rate': 0.3534, 'clf_aucroc': 0.9152}

Fold 3
Generated incomplete data with missing values for fold 3:
Original data with missing values (subset):
[[0 0 0 22 1 0 7]
 [1 1 1 38 nan 0 71]
 [1 0 1 26 0 0 8]
 [0 0 0 nan 0 0 8]
 [0 0 0 27 0 0 8]]


...

Completed imputation for fold 3:
X_res with imputed values (subset):
[[0 0 0 22 1 0 7]
 [1 2 1 38 1 0 71]
 [1 0 1 26 0 0 8]
 [0 0 0 27.635210037231445 0 0.3982674777507782 8]
 [0 0 0 27 1.3875157833099365 0 8]]
Metrics for this fold: {'nrmse': 0.7478, 'col_0_error_rate': 0.3986, 'col_1_error_rate': 0.6579, 'col_2_error_rate': 0.2857, 'total_error_rate': 0.4591, 'clf_aucroc': 0.7915}

Fold 4
Generated incomplete data with missing values for fold 4:
Original data with missing values (subset):
[[0 0 0 22 1 0 7]
 [nan 1 1 38 nan nan 71]
 [nan nan nan 35 1 0 53]
 [0 nan 0 35 0 0 8]
 [0 0 0 27 0 0 8]]


100%|███████████████████████████████████████████| 15/15 [00:04<00:00,  3.46it/s]


Completed imputation for fold 4:
X_res with imputed values (subset):
[[0 0 0 22 1 0 7]
 [1 1 1 38 2.020275592803955 2.527261734008789 71]
 [1 0 0 35 1 0 53]
 [0 0 0 35 0 0 8]
 [0 0 0 27 0 0 8]]
Metrics for this fold: {'nrmse': 0.763, 'col_0_error_rate': 0.4551, 'col_1_error_rate': 0.4715, 'col_2_error_rate': 0.2532, 'total_error_rate': 0.388, 'clf_aucroc': 0.8284}

Fold 5
Generated incomplete data with missing values for fold 5:
Original data with missing values (subset):
[[0 0 0 22 1 0 7]
 [1 1 1 38 1 0 71]
 [1 0 1 26 nan 0 8]
 [1 1 1 35 1 nan nan]
 [nan 0 0 27 0 0 nan]]


100%|███████████████████████████████████████████| 15/15 [00:03<00:00,  3.99it/s]

Completed imputation for fold 5:
X_res with imputed values (subset):
[[0 0 0 22 1 0 7]
 [1 1 1 38 1 0 71]
 [1 0 1 26 1.3410472869873047 0 8]
 [1 1 1 35 1 0.40731361508369446 66.6235122680664]
 [0 0 0 27 0 0 18.985668182373047]]
Metrics for this fold: {'nrmse': 0.856, 'col_0_error_rate': 0.3385, 'col_1_error_rate': 0.561, 'col_2_error_rate': 0.381, 'total_error_rate': 0.4225, 'clf_aucroc': 0.8455}

Fold 6
Insufficient data for validation, breaking loop
Average nrmse for 20% missing data: 0.9320599999999999
Average col_0_error_rate for 20% missing data: 0.38452000000000003
Average col_1_error_rate for 20% missing data: 0.5393399999999999
Average col_2_error_rate for 20% missing data: 0.31126
Average total_error_rate for 20% missing data: 0.41372
Average clf_aucroc for 20% missing data: 0.8430000000000002
Completed all iterations
